In [5]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from xgboost import XGBClassifier, XGBRegressor

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df=pd.read_csv('data/stud.csv')